In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import pickle
import os
import traceback

# ===============================
# CONFIGURATION
# ===============================

MODEL_DIR = "models"

BUY_THRESHOLD = 0.55
SELL_THRESHOLD = 0.45

stocks = [
    "RELIANCE.NS", "HDFCBANK.NS", "TCS.NS", "INFY.NS",
    "ICICIBANK.NS", "SBIN.NS", "BHARTIARTL.NS",
    "BAJFINANCE.NS", "HINDUNILVR.NS", "LT.NS"
]

signals = []

# ===============================
# MAIN LOOP
# ===============================

for ticker in stocks:

    print(f"\nProcessing {ticker}")
    stock_name = ticker.replace(".NS", "")

    try:
        # -----------------------------
        # Load Model Files from models/
        # -----------------------------
        model_path = os.path.join(MODEL_DIR, f"{stock_name}_model.pkl")
        scaler_path = os.path.join(MODEL_DIR, f"{stock_name}_scaler.pkl")
        feature_path = os.path.join(MODEL_DIR, f"{stock_name}_features.pkl")

        model = pickle.load(open(model_path, "rb"))
        scaler = pickle.load(open(scaler_path, "rb"))
        feature_columns = pickle.load(open(feature_path, "rb"))

        # Fix old tuple features if needed
        feature_columns = [
            col[0] if isinstance(col, tuple) else col
            for col in feature_columns
        ]

        # -----------------------------
        # Download Latest Data
        # -----------------------------
        data = yf.download(
            ticker,
            start="2015-01-01",
            auto_adjust=True,
            progress=False
        )

        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        # -----------------------------
        # Feature Engineering
        # -----------------------------
        data["Return"] = data["Close"].pct_change()
        data["SMA_5"] = data["Close"].rolling(5).mean()
        data["SMA_20"] = data["Close"].rolling(20).mean()
        data["EMA_10"] = data["Close"].ewm(span=10).mean()
        data["Volatility"] = data["Return"].rolling(10).std()
        data["Momentum"] = data["Close"] - data["Close"].shift(5)

        for i in range(1, 6):
            data[f"Lag_{i}"] = data["Return"].shift(i)

        data.dropna(inplace=True)

        if len(data) == 0:
            print("No usable rows")
            continue

        # -----------------------------
        # Prediction
        # -----------------------------
        X_live = data[feature_columns].iloc[[-1]]
        X_live_scaled = scaler.transform(X_live)

        probability = model.predict_proba(X_live_scaled)[0][1]

        # -----------------------------
        # Probability-Based Signal Logic
        # -----------------------------
        if probability > BUY_THRESHOLD:
            signal = "BUY"
        elif probability < SELL_THRESHOLD:
            signal = "SELL"
        else:
            signal = "HOLD"

        signals.append({
            "Stock": ticker,
            "Probability": round(probability, 4),
            "Confidence (%)": round(probability * 100, 2),
            "Signal": signal
        })

    except Exception:
        print(f"\n Error processing {ticker}")
        traceback.print_exc()
        continue

signals_df = pd.DataFrame(signals)

if len(signals_df) > 0:
    signals_df = signals_df.sort_values("Probability", ascending=False)
    print("\n========== PROBABILITY BASED STOCK SIGNALS ==========\n")
    display(signals_df)
else:
    print("No signals generated.")


Processing RELIANCE.NS

Processing HDFCBANK.NS

Processing TCS.NS

Processing INFY.NS

Processing ICICIBANK.NS

Processing SBIN.NS

Processing BHARTIARTL.NS

Processing BAJFINANCE.NS

Processing HINDUNILVR.NS

Processing LT.NS

========== PROBABILITY BASED STOCK SIGNALS ==========



,Stock,Probability,Confidence (%),Signal
8,HINDUNILVR.NS,0.1816,18.16,SELL
0,RELIANCE.NS,0.1252,12.52,SELL
6,BHARTIARTL.NS,0.1055,10.55,SELL
1,HDFCBANK.NS,0.0704,7.04,SELL
3,INFY.NS,0.0668,6.68,SELL
2,TCS.NS,0.0456,4.56,SELL
4,ICICIBANK.NS,0.0234,2.34,SELL
7,BAJFINANCE.NS,0.0096,0.96,SELL
9,LT.NS,0.0062,0.62,SELL
5,SBIN.NS,0.0047,0.47,SELL


In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
import pickle
import os
import traceback

# =====================================
# CONFIGURATION
# =====================================

MODEL_DIR = "models"

BUY_THRESHOLD = 0.55
SELL_THRESHOLD = 0.45

stocks = [
    "RELIANCE.NS", "HDFCBANK.NS", "TCS.NS", "INFY.NS",
    "ICICIBANK.NS", "SBIN.NS", "BHARTIARTL.NS",
    "BAJFINANCE.NS", "HINDUNILVR.NS", "LT.NS"
]

signals = []

# =====================================
# PREDICTION LOOP
# =====================================

for ticker in stocks:

    stock_name = ticker.replace(".NS", "")

    try:
        # Load model files from models/
        model_path = os.path.join(MODEL_DIR, f"{stock_name}_model.pkl")
        scaler_path = os.path.join(MODEL_DIR, f"{stock_name}_scaler.pkl")
        feature_path = os.path.join(MODEL_DIR, f"{stock_name}_features.pkl")

        model = pickle.load(open(model_path, "rb"))
        scaler = pickle.load(open(scaler_path, "rb"))
        feature_columns = pickle.load(open(feature_path, "rb"))

        # Fix tuple feature names (if any)
        feature_columns = [
            col[0] if isinstance(col, tuple) else col
            for col in feature_columns
        ]

        # Download latest data
        data = yf.download(
            ticker,
            period="6mo",
            auto_adjust=True,
            progress=False
        )

        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        # Feature Engineering
        data["Return"] = data["Close"].pct_change()
        data["SMA_5"] = data["Close"].rolling(5).mean()
        data["SMA_20"] = data["Close"].rolling(20).mean()
        data["EMA_10"] = data["Close"].ewm(span=10).mean()
        data["Volatility"] = data["Return"].rolling(10).std()
        data["Momentum"] = data["Close"] - data["Close"].shift(5)

        for i in range(1, 6):
            data[f"Lag_{i}"] = data["Return"].shift(i)

        data.dropna(inplace=True)

        if len(data) == 0:
            continue

        # Prediction
        X_live = data[feature_columns].iloc[[-1]]
        X_live_scaled = scaler.transform(X_live)

        probability = model.predict_proba(X_live_scaled)[0][1]

        # Signal Logic
        if probability > BUY_THRESHOLD:
            signal = "BUY"
        elif probability < SELL_THRESHOLD:
            signal = "SELL"
        else:
            signal = "HOLD"

        signals.append({
            "Stock": ticker,
            "Probability": round(probability, 4),
            "Confidence (%)": round(probability * 100, 2),
            "Signal": signal
        })

    except Exception:
        print(f"\n Error processing {ticker}")
        traceback.print_exc()
        continue


# =====================================
# CLEAN SIGNAL TABLE
# =====================================

signals_df = pd.DataFrame(signals)

if len(signals_df) > 0:

    # Sort by highest probability
    signals_df = signals_df.sort_values("Probability", ascending=False).reset_index(drop=True)

    print("\n========== TODAY'S STOCK SIGNAL TABLE ==========\n")
    display(signals_df)

else:
    print("\nNo signals generated.")


========== TODAY'S STOCK SIGNAL TABLE ==========



,Stock,Probability,Confidence (%),Signal
0,HINDUNILVR.NS,0.1816,18.16,SELL
1,RELIANCE.NS,0.1252,12.52,SELL
2,BHARTIARTL.NS,0.1055,10.55,SELL
3,HDFCBANK.NS,0.0704,7.04,SELL
4,INFY.NS,0.0668,6.68,SELL
5,TCS.NS,0.0456,4.56,SELL
6,ICICIBANK.NS,0.0234,2.34,SELL
7,BAJFINANCE.NS,0.0096,0.96,SELL
8,LT.NS,0.0062,0.62,SELL
9,SBIN.NS,0.0047,0.47,SELL
